In [3]:
from pythainlp.tokenize import word_tokenize
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

from gensim.models import KeyedVectors
import numpy as np

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

import dill as pickle
#import pandas as pd

DATA_PATH='../lm_data/'
MODEL_PATH = f'{DATA_PATH}models/'
MISC_PATH = f'{DATA_PATH}misc/'

In [4]:
model = KeyedVectors.load_word2vec_format(f'thai2vec.bin',binary=True)
words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [5]:
thai_letters = 'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤฤๅลฦฦๅวศษสหฬอฮะัาำิีึืุูเแโใไ็่้๊๋์'


def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    
    if word==" " or word=='':
        return('')
    else:
   
        return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = thai_letters
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [8]:
print(candidates('พัดนา'))
print(correction('พัดนา'))
print(candidates('จย้า'))
print(candidates('ขริง'))
print(correction('ขริง'))
print(candidates('วั'))
print(correction('วั'))
print(candidates('บุรุก'))
print(correction('บุรุก'))
print(correction(' '))
print(correction('ได่'))

{'พัดพา', 'พัฒนา', 'พัดชา'}
พัฒนา
{'จ้า', 'หย้า', 'เย้า'}
{'ขิง', 'ขริบ', 'จริง', 'เริง'}
จริง
{'วัฒ', 'ทั', 'วัต', 'วน', 'วัร', 'วว', 'ว๋', 'วู', 'วล', 'ั', 'วซ', 'วจ', 'ปั', 'วัม', 'วิ', 'วัช', 'วัศ', 'ลั', 'วัจ', 'วัย', 'วี', 'วต', 'ว้', 'ตั', 'วย', 'ดั', 'ษั', 'วม', 'วง', 'วัซ', 'วัล', 'วบ', 'ขั', 'วท', 'วัธ', 'วัฏ', 'วัง', 'วับ', 'ว์', 'วร', 'จั', 'พั', 'วำ', 'วะ', 'วด', 'วัท', 'วัด', 'วัณ', 'นั', 'วพ', 'วัว', 'วัค', 'กั', 'วอ', 'วัน', 'วป', 'วส', 'ว่', 'วัญ', 'รั', 'วา', 'ว', 'วก', 'ัว', 'วัส', 'วัฟ', 'วัก', 'วุ', 'วห', 'วัฎ'}
วัน
{'บุรุษ', 'บุกรุก'}
บุรุษ

ได่


In [151]:
import time

start = time.time()
read_text = open('politics_output.txt','r',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    print(correction(i))
read_text.close()
end = time.time()
print(end-start)

ปลื้ม
เตรียม
ร่วม
ประชาธิปัตย์
เผย
พ่อ
สนับสนุน
สุรบถหลีก
ภัย
เตรียม
สมัคร
สมาชิก
พรรคประชาธิปัตย์
เผย
พร้อม
ลง
ส.ส.
แบบ
แบ่ง
เขต
-
ปาร์ตี้
ลิฟต์
3.5446949005126953


In [145]:
import time

start = time.time()
read_text = open('1_ตัด.txt','r',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    print(correction(i))
read_text.close()
end = time.time()
print(end-start)

รอง
ส
ผบช.น.
ส
เผย
ส
ส
ปัณณวิชญ์
ส
ประชุม
ส
ส
หัวหน้า
แก๊ง
ยา
เสพ
ติด
มัน
ทุก
เม็ด
ส
คือ
หนึ่ง
ใน
แอดมินเพจ
ส
ส
เทพบุตร
ส
โซโล
ส
ส
หลบหนี
อยู่
ใน
ประเทศ
เพื่อนบ้าน
ส
คาด
ตั้งแต่
เริ่ม
ค้า
ยา
ส
ขาย
ยา
เสพ
ติด
ไป
มาก
กว่า
ส
1
,
000
ส
ล้าน
ใน
วัน
ที่
ส
23
ส
ม.ค.
ส
พล.ต.ต.พรชัย เจริญวงศ์
ส
รอง
ผู้
บัญชา
การ
ตำรวจปราบปราม
ยา
เสพ
ติด
ส
เปิดเผย
ว่า
ส
เทพ โซโล
ส
เป็น
บัญชี
ผู้
ใช้
เฟซ
บุ๊ก
ซึ่ง
มี
ผู้
ค้า
ยา
เสพ
ติด
หมุนเวียน
เข้า
ไป
ใช้
งาน
ไม่
ต่ำ
กว่า
ส
4
ส
คน
ส
และ
หนึ่ง
ใน
นั้น
คือ
ส
นายปัณณวิชญ์ 
ประชุม
ส
หัวหน้า
กลุ่ม
ค้า
ยา
เสพ
ติด
มัน
ทุก
เม็ด
ส
ที่
อยู่
ระหว่าง
การ
หลบหนี
ใน
ฝั่ง
ประเทศ
เพื่อนบ้าน
ส
ซึ่ง
จาก
การ
ตรวจสอบ
บัญชี
ดัง
กล่าว
ส
มี
ความ
เชื่อมโยง
ไป
ยัง
ผู้
ใช้
บัญชี
เฟซ
บุ๊กเครือข่าย
ค้า
ยา
เสพ
ติด
อีก
กว่า
ส
20
ส
บัญชี
ส
และ
จาก
การ
ตรวจสอบ
พยาน
หลักฐาน
ที่
รวบรวม
ได้
ส
พบ
ว่า
ปัจจุบัน
กลุ่ม
มัน
ทุก
เม็ด
ถูก
จับกุม
ไป
แล้ว
ส
30
ส
คน
ส
ยัง
เหลือ
อีก
ส
9
ส
คน
ส
ที่
อยู่
ระหว่าง
การ
หลบหนี
ส
ใน
จำนวน
นี้
มี
ผู้
ไป
เปิด
บัญชี
เฟซ
บุ๊ก
ใน
ชื่อ
ส
เทพบุตร
ส
โซโล
ส
ใช้
แทน
บัญชี
เ

In [148]:
import time

start = time.time()
read_text = open('2_ตัด.txt','r',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    print(correction(i))
read_text.close()
end = time.time()
print(end-start)

โหดร้าย
ส
เผย
นาที
ชาย
ชุด
ดำ
บุก
วัด
ส
จ่อ
ยิง
เจ้าคณะ
อำเภอ
ส
พร้อม
ลูก
วัด
ส
4
ส
ศพ
โดย
ส
ไทย
รัฐ
ออนไลน์
18
ส
ม.ค.
ส
2562
ส
22
:
18
ส
น.
เผยนาที
สังหาร
พระ
ที่นราธิวาส
ส
คน
ร้าย
ชุด
ดำ
ส
ขี่
จ.
ทุกวัน
ส
ส
ใช้
อาวุธ
ปืน
ยิง
เจ้าคณะ
อำเภอสุไหงปาดี
ส
มรณภาพ
พร้อม
ลูก
วัด
4
รูป
โยม
สุด
สลด
ส
เห็น
กับ
ตา
ส
พระ
ถูก
จ่อ
ยิง



จาก
เหตุ
คน
ร้าย
ยิง
ใส่
วัด
ส
ที่
อ.สุไหงปาดี
ส
นราธิวาส
ส
ผู้
สื่อ
ข่าว
รายงาน
ว่า
ส
เกิด
ขึ้น
ใน
เวลา
ส
20
.
30
ส
น.
ส
วัน
ที่
ส
18
ส
ม.ค.
ส
62
ส
ได้
มี
คน
ร้าย
จำนวน
กว่า
ส
10
ส
คน
ส
แต่ง
กาย
ชุด
ดำ
คล้าย
ทหาร
ส
ขี่
รถ
จ.
เป็น
พาหนะ
ส
บุก
เข้า
ไป
ภาย
ใน
วัดรัตนานุภาพ
ส
หรือ
ส
วัดโคกโก
ส
ซึ่ง
ตั้ง
อยู่
บ้านโคกโก
ส
ม.
2
ส
ต.โต๊ะเด็ง
ส
อ.สุไหงปาดี
ส
นราธิวาส
ส
แล้ว
ได้
กระจาย
กำลัง
ใช้
อาวุธ
ปืน
สงคราม
ยิง
ถล่ม
ใส่
กุฏิพระ
ส
ส่งผล
ทำให้
พระครูประโชติรัตนานุรักษ์
ส
เจ้าคณะ
อำเภอสุไหงปาดี
ส
และ
เป็น
เจ้า
อาวาส
วัดรัตนานุภาพ
ส
มรณภาพ
พร้อม
พระลูกวัด
ส
รวม
ส
3
ส
รูป
ส
และ
มี
พระ
ลูกปัด
ได้
รับ
บาดเจ็บ
อีก
ส
1
ส
รูป
ขณะ
เกิด
เหตุ
ส
ชาว
บ้าน
ที่
อาศัย
อยู่
บริเวณ
ด้าน
นอ

In [152]:
start = time.time()
read_text = open('politics_output.txt','r',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    x = [y.replace(i,(correction(i))) for y in x]
    new_text = open('newpolitics2.txt','w',encoding='utf-8')
    text2 = new_text.write("|".join(x))

end = time.time()
print(end-start)

55.701732873916626


In [61]:
import time

start = time.time()
read_text = open('print.txt','r',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
        x = [y.replace(i,(correction(i))) for y in x]
        open('newprint.txt','w',encoding='utf-8').write("|".join(x))
        
end = time.time()
print(end-start)

7.6693434715271


In [95]:
import time

start = time.time()
read_text = open('politics_output.txt','r+',encoding='utf-8')
text = read_text.read()
x = text.split("|")

out_text = open('newpolitics.txt','w+',encoding='utf-8')

#print(x[0])
for i in x:
        x = [y.replace(i,(correction(i))) for y in x]
        output = ("|".join(x))

print(output, file=out_text)        
out_text.close()

end = time.time()
print(end-start)

72.46736884117126


In [101]:
import time

start = time.time()
read_text = open('politics_output.txt','r',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
        x = [y.replace(i,(correction(i))) for y in x]
        open('newpolitics.txt','w',encoding='utf-8').write("|".join(x))
        
end = time.time()
print(end-start)

86.5016655921936


In [160]:
import time

start = time.time()
read_text = open('news_2_ตัด.txt','r+',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
        print(correction(i))
        
        
end = time.time()
print(end-start)


ไม้
หน้า
สาม
ฟาดหัว
กิ๊ก
สาว
ดับ

เฒ่า
วัย

72

หนี
ผิด
ซด
ยา
ฆ่า
แมลง

แขวน
สยอง
ส
โดย

ไทย
รัฐ
ออนไลน์
22

ม.ค.

2562

01
:
27

น.
ส
ฆ่า
กิ๊ก
สาว
วัย

26

ไม้
หน้า
สาม
ฟาด
หัวสูง
ที่
เชียงใหม่

เฒ่า
วัย

72

กลัว
ความ
ผิด
ซด
เหล้า
ผสม
ยา
ฆ่า
แมลง

ก่อน
แขวน
คอ
ตาย
ตาม
ใน
ห้อง
นอน

ตร.
คาด
ชู้
สาว

เชื่อ
มี
การ
นัดแนะ
ให้
มา
พบ
กัน

และ
เกิด
มี
ปากเสีย
ง
...
ส
ส
ส
เมื่อ
เวลา

20
.
00

น.

วัน
ที่

21

ม.ค.
62

ร.ต.อ.ทินกร โชติชัยพิบูล

รอง

สวน
(
สอบสวน
)

สภ.สันกำแพง

เชียงใหม่

ได้
รับ
แจ้ง
มี
เหตุ
ฆ่า
กัน
และ
แขวน
คอ
ตาย

ที่
ห้อง
แถว

หมู่
บ้านแพวาสัน
ป่าช้า

เลข
ที่

147
/
48

หมู่

8

ต.ต้นเปา

อ.สันกำแพง

จึง
รุด
ไป
ตรวจสอบ
ที่
เกิด
เหตุ

พร้อม
ด้วย

พ.ต.อ.ชูวิทย์ กองแก้ว

ผกก.

พ.ต.ท.ถาวร นันติแสน

รอง

ผกก.
(
สอบสวน
)

พ.ต.ท.คมสันต์ บำรุงยศ

รอง

ผกก.
สส.
พร้อม
ชุด
สืบสวน

และ
แพทย์
เวร

รพ.สันกำแพง

ด.ต.สมบูรณ์ ชัยชมภู

ผบ.
เจ้าของ
พื้นที่
ส
ส
ที่
เกิด
เหตุ
เป็น
บ้าน
ชั้น
เดียว

กว้าง
ประมาณ

4

เมตร

ภาย
ใน
ห้อง
โถง

พบ
ขวด
หล้า
และ
ชาม
ก๋วยเตี๋ยว

น้ำ
อัด
ลม

กระติก
น้ำแข

In [5]:
import time

start = time.time()
read_text = open('news/news_1_ตัด.txt','r+',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
        print(correction(i))
        
        
end = time.time()
print(end-start)


จับ
2
หนุ่ม

โชว์
ปืน
ขู่
เด็ก
ปั๊ม

แถม
ดูด
กัญชา

โดน
ไป

4

ข้อ
หา
ส
โดย

ไทย
รัฐ
ออนไลน์
27

ม.ค.

2562

14
:
38

น.
ส
hare
:
ส
สอง
หนุ่ม
คู่หู

มี
ปืน
อยู่

1

กระบอก

โชว์
ขู่
เด็ก
ปั๊ม

ก่อน
ไป
ทำ
แบบ
เดียว
กัน
กับ
โจ๋
ที่
จับ
กลุ่ม
กัน
อยู่

โดน

191

ตาม
จับ

สารภาพ
เพิ่ง
ดูด
กัญชา
มา

เลย
โดน
จับกัญชา
พ่วง
ไป
ด้วย
ส
ส
เมื่อ
เวลา

12
.
00

น.

วัน
ที่

27

ม.ค.

ที่
ห้อง
ปฏิบัติการ
งาน
สาย
ตรวจ
3

หอบังคับการ
สาย
ตรวจ
และ
ปฏิบัติการ
พิเศษ
191
(
บก.สปพ.
)

ถนนวิภาวดี

พล.ต.ต.สำราญ นวลมา

ผบก.สปพ.

พร้อม
ด้วย

พ.ต.อ.สมบูรณ์ เทียนขาว

ผกก.
สาย
ตรวจ

บก.สปพ.

พ.ต.ท.อานันท์จักร์ กนกนพวัชร์

รอง

ผกก.
สาย
ตรวจ

ร่วม
กัน
แถลง
ผล
การ
จับกุม

นายแบ้ง ทองแท้

อายุ

25

ปี

ชาว
จ.
ปราจีนบุรี

และ

นายกฤษฎา โป๊ะแก้ว

อายุ

28

ปี

ชาว

ชลบุรี

โดย
สามารถ
จับกุม
ได้
บริเวณ
หน้า
การ
ประปา
ภาษีเจริญ

แขวงบางแค
เหนือ

เขตบางแค

กทม.

พร้อม
ของกลาง
อาวุธ
ปืน
ลูก
ซอง
สั้น
ไทย
ประดิษฐ์

กระสุน
ปืน
ลูก
ซอง

เบอร์
12

จำนวน

1

นัด

กัญชา
น้ำหนัก
ประมาณ

20

กรัม

อุปกรณ์
การ
เสพ
กัญชา

และ

รถยน

In [67]:
import time

start = time.time()
read_text = open('news/news_3_ตัด.txt','r+',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    if i != ' ' and i != '' and i != '\n':    
        print(correction(i))

end = time.time()
print(end-start)

รู้
ตัว
คน
ร้าย
ฆ่า
หั่น
ศพ
หนุ่ม
เกาหลีใต้
พบ
เป็น
คน
ร่วม
ชาติ
ขัดแย้ง
ธุรกิจ
มืด
โดย
ไทย
รัฐ
ออนไลน์
24
ม.ค.
2562
10
:
33
น.
หนุ่ม
โปรแกรม
เมอร์วัย
32
ปี
พร้อม
เจ้าหน้าที่
สถาน
ทูต
เกาหลีใต้
มา
มอบ
ตัว
ถึง
กองปราบ
หลัง
ร่วม
อยู่
ใน
เหตุการณ์
เพื่อน
ร่วม
ชาติ
ถูก
ฆ่า
หั่น
ศพ
นำ
ชิ้นส่วน
ทิ้ง
ป่า
ใน
ระยอง
เผย
ถูก
บังคับ
ให้
นั่ง
ดู
ถ้า
ไม่
ดู
จะ
กลาย
เป็น
ศพ
แทน
จาก
กรณี
ที่
มี
ชาว
บ้าน
พบ
ชิ้นส่วน
มนุษย์
ที่
นำ
มา
ทิ้ง
ไว้
บริเวณ
ป่า
ส่วนแบ่ง
ชาติหนองสนม
จังหวัดระยอง
โดย
ชิ้นส่วน
ดัง
กล่าว
น่า
จะ
เป็น
นายชเว มยองฮุน ชื่อ
(
CHOI
MYONGHOON
)
อายุ
35
ปี
ชาว
เกาหลีใต้
ตาม
ที่
ได่
เสนอ
ข่าว
ไป
ก่อนหน้า
นี้
(
รู้
แล้ว
ฆ่า
หั่น
ศพ
ยัด
ถุง
ดำ
ทิ้ง
ระยอง
เป็น
หนุ่ม
เกาหลีใต้
)
ล่าสุด
เมื่อ
เวลา
09
.
30
น.
วัน
ที่
24
ม.ค.
61
ผู้
สื่อ
ข่าว
รายงาน
ว่า
ช่วง
กลาง
ดึก
ที่
ผ่าน
มา
เจ้าหน้าที่
สถาน
เอกอัครราชทูต
สาธารณรัฐ
เกาหลีประจำ
ประเทศไทย
ได้
นำ
ตัว
นายคิม ฮยอนจุน
(
im
Hyeonjun
)
อายุ
32
ปี
ผู้ต้องหา
ร่วม
กัน
ฆ่า
หั่น
ศพ
นายชเว มยองฮุน
เดินทาง
เข้า
มอบ
ตัว
กับ
พล.ต.ต.จิรภพ ภูริเดช
ผบก.ป.พ.ต.อ.อร

In [70]:
import time

start = time.time()
read_text = open('news/news_4_ตัด-2.txt','r+',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    if i != ' ' and i != '' and i != '\n':    
        print(correction(i))

end = time.time()
print(end-start)

จับ
ลูก
ชาย
วนิษฐา วัชโรบล ดารา
รุ่น
ใหญ่
ยิง
อริ
จับ
บั้นท้าย
เพื่อน
สาว
ใน
ผับ
โดย
ไทย
รัฐ
ออนไลน์
20
ม.ค.
2562
17
:
25
น.
กองปราบ
จับ
ลูก
ชาย
วนิษฐา วัชโรบล ดารา
รุ่น
ใหญ่
ละคร
จักร
ๆ
วงศ์
ๆ
ตาม
หมาย
หลัง
ร่วม
ยิง
อริ
ที่
จับ
บั้นท้าย
เพื่อน
สาว
ใน
ผับ
ที่
ปาก
ช่อง
เจ้าตัว
ปฏิเสธ
รับ
มี
เรื่อง
จริง
แต่
ออก
จาก
ผับ
ก็
แยกย้าย
...
วัน
ที่
20
ม.ค.
62
ที่
หอบังคับการ
ปราบปราม
พล.ต.ต.จิรภพ ภูริเดช
ผบช.น.
สั่งการ
ให้
พ.ต.อ.บุญลือ ผดุงถิ่น
ผกก.
3
ก.พ.
พ.ต.ท.สุริยศักดิ์ จิราวัสน์
สวน
กก.
ก.พ.
พ.ต.ท.ภูมิทัศ ปิติ
จิระนคร
สวน
กก.
ก.พ.
นำ
กำลัง
จับกุม
นายณัฎฐชัย บุญญาหาร
อายุ
37
ปี
ลูก
ชาย
ของ
นางวนิษฐา วัชโรบล
นัก
แสดง
ละคร
จักรๆ วงศ์
ๆ
รุ่น
ใหญ่
ตาม
หมาย
จับ
ศาล
สี
คิ้ว
ที่
54
/
2555
ฐาน
ความ
ผิด
“
ร่วม
กัน
ฆ่า
ผู้
อื่น
โดย
เจตนา
พยายาม
ฆ่า
ผู้
อื่น
และ
พา
อาวุธ
ปืน
ติด
ตัว
ไป
ใน
เมือง
หมู่
บ้าน
หรือ
ทาง
สาธารณะ
โดย
ไม่
ได้
รับ
อนุญาต
”
โดย
จับ
ได้
บริเวณ
หน้า
บ้าน
พัก
ของ
ผู้ต้องหา
ภาย
ใน
ซอยโชค
ชัย 4

พ.ต.อ.บุญลือ
กล่าว
ว่า
ก่อนหน้า
นี้


KeyboardInterrupt: 

In [13]:
import time

start = time.time()
read_text = open('news/test.txt','r+',encoding='utf-8')
text = read_text.read()
x = text.split("|")

#print(x[0])
for i in x:
    if i != ' ' and i != '' and i != '\n':    
        print(correction(i))

end = time.time()
print(end-start)

กรี๊ด
พี่
เวย์
ไท
เท
เนียม
นะ
ค่ะ
0.017002582550048828
